# Import data

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
traindf, testdf = train_test_split(pd.read_csv('github_issues.csv').sample(n=2000000), 
                                   test_size=.10, random_state=0)

#print out stats about shape of data
print('Train: {} rows {} columns'.format(traindf.shape[0],traindf.shape[1]))
print('Test: {} rows {} columns'.format(testdf.shape[0],testdf.shape[1]))

traindf.head()

Train: 1800000 rows 3 columns
Test: 200000 rows 3 columns


,issue_url,issue_title,body
4457408,"""https://github.com/circlecell/jsonlintpro/iss...",long num changed after validation,"{ i : 4012441203977093419,4012441203977093420 ..."
33119,"""https://github.com/jiggzson/nerdamer/issues/161""",acosh 1/23.12 throws,"sorta related to 157. this, instead of not eva..."
1905639,"""https://github.com/truckerpath/clj-datadog/is...",system/currenttimemillis vs system/nanotime,would it be better to use nanotime here instea...
1774158,"""https://github.com/BeyondTrust/pbis-open/issu...",cannot send different fqdn to ad,version : any os/distro : any issue/impact : c...
3703906,"""https://github.com/Automattic/facebook-instan...",no rules defined for <div class= entry-content...,no rules defined for <div class= entry-content...


In [3]:
train_body_raw = traindf.body.tolist()
train_title_raw = traindf.issue_title.tolist()

# Preprocessing

#### For x (body)

In [4]:
from keras.preprocessing.text import Tokenizer

tk_body = Tokenizer()
tk_body.fit_on_texts(train_body_raw)
tk_body.num_words = 8000
train_body_vecs = tk_body.texts_to_sequences(train_body_raw)

/home/richard/deep_learn_p3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
from keras.preprocessing.sequence import pad_sequences
train_body_vecs = pad_sequences(train_body_vecs, 70)

#### For y (title)

In [19]:
from Helpers import custom_pad_sequences

train_title_vecs, tk_title, _, _ = custom_pad_sequences(train_title_raw, 4500, 12)

tokenizing...
padding...


#### Save tokenizers and vecs to local

In [20]:
import dill as dpickle
import numpy as np

# Save the preprocessor
with open('tk_body.dpkl', 'wb') as f:
    dpickle.dump(tk_body, f)

with open('tk_title.dpkl', 'wb') as f:
    dpickle.dump(tk_title, f)

# Save the processed data
np.save('train_title_vecs.npy', train_title_vecs)
np.save('train_body_vecs.npy', train_body_vecs)